# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [13]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [14]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [15]:
def get_numpy_data(data_sframe,features,output):
    # add a constant column to an SFrame 
    data_sframe['constant'] =1 
    features = ['constant'] + features 
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe =data_sframe[features]
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray=data_sframe[output]
    # this will convert the SArray into a numpy array
    output_array = output_sarray.to_numpy()
    return(features_matrix,output_array)


Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [16]:
def predict_output(feature_matrix,weights):
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [17]:
x=np.array([[  1.00000000e+00,   1.18000000e+03,   3.00000000e+00]])
y=np.array([1., 4., 1.])
pred = predict_output(x,y)
pred

array([ 4724.])

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [18]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [19]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms
# axis =0 means col wise sqrt(3^2 + 4^2), sqrt(5^2 + 12^2 ) ,sqrt(8^2 + 15^2 )
# axis =1 means rows wise sqrt(3^2+5^2+8^2), sqrt(4^2+12^2+15^2)

[  5.  13.  17.]


To normalize, apply element-wise division:

In [20]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]
# [3/5 ,5/13,8/17 ] so is for 2nd row

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [21]:
def normalize_features(feature_matrix):
    #print feature_matrix
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = (feature_matrix / norms )
    return (normalized_features,norms)
    

To test the function, run the following:

In [22]:
feature_matrix=np.array([[3.,6.,9.],[4.,8.,12.]])
features,norms = normalize_features(feature_matrix)
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
#[  5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


In [23]:
feature_matrix=np.array([[3.,6.,9.],[4.,8.,12.]])
norms = np.linalg.norm(feature_matrix,axis=0)
normalized_features = feature_matrix/norms 
print normalized_features
print ''
print norms


[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]

[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [24]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [25]:
simple_feature_matrix

array([[  1.00000000e+00,   1.18000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   2.57000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   7.70000000e+02,   2.00000000e+00],
       ..., 
       [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00],
       [  1.00000000e+00,   1.60000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00]])

Don't forget to normalize features:

In [26]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [27]:
simple_feature_matrix

array([[ 0.00680209,  0.00353021,  0.00583571],
       [ 0.00680209,  0.00768869,  0.00583571],
       [ 0.00680209,  0.00230361,  0.00389048],
       ..., 
       [ 0.00680209,  0.00305154,  0.00389048],
       [ 0.00680209,  0.00478673,  0.00583571],
       [ 0.00680209,  0.00305154,  0.00389048]])

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [34]:
weights = np.array([1., 4., 1.])
weights[1]

4.0

Use `predict_output()` to make predictions on this data.

In [29]:
prediction = predict_output(simple_feature_matrix,weights)

In [30]:
prediction

array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

In [31]:
simple_feature_matrix[:,0] # all rows 0th col ,here we are extracting the features which is cols 
simple_feature_matrix[:,1] # all rows 1st col 
simple_feature_matrix.shape[1] # no. of cols ie no. of features 

3

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [54]:
#Note that we do not regularize the weight of the constant feature (intercept) w[0], 
#so, for this weight, the update is simply: w0 =ro[i]
num_features = simple_feature_matrix.shape[1]
# print len(simple_features)
r0={}
for i in range(num_features):
    if(i==0):
        r0[i]=weights[0]
    else:
        r0[i]=(simple_feature_matrix[:,i]*(output - prediction + (weights[i]*simple_feature_matrix[:,i]))).sum()

In [55]:
r0 #{0: 1.0, 1: 87939470.772991076, 2: 80966698.675965652}

{0: 1.0, 1: 87939470.772991076, 2: 80966698.675965652}

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [57]:
# w[i]=0 ,if -lambda/2 <= ro[i] <= lambda/2
# so for w[2] =0 , multiply 2 on both sides of above i.e -lambda <= 2*ro[i] <=lambda 
# i.e lambda >= -2*ro[i] and lambda <=2*ro[i] i.e -2*ro[i] <= lambda <= 2*ro[i]
print (-2*r0[2],2*r0[2] )


# What range of values of l1_penalty would not set w[1] zero
'''
      ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
'''
# for lambda not to be zero ro[i] < -lambda/2 i.e -2*ro[i] > lambda  and ro[i] > lambda/2 i.e 2*ro[i] > lambda
print (-2*r0[1],2*r0[1])
# i.e -175878941.54598215 > stricty greater than lambda and 175878941.54598215  > lambda 

(-161933397.3519313, 161933397.3519313)
(-175878941.54598215, 175878941.54598215)


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [58]:
print (-2*r0[1],2*r0[1] )
print (-2*r0[2],2*r0[2] )
# for both of them to be zero lambda shd be within the above ranges 

(-175878941.54598215, 175878941.54598215)
(-161933397.3519313, 161933397.3519313)


'''
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
'''

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [64]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i]*(output - prediction + (weights[i]*feature_matrix[:,i]))).sum()
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i =  ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [65]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                                                 [2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [100]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged=False
    while not converged:
        weights_diff=[]
        for i in range(len(initial_weights)):
            old_weights_i = initial_weights[i]
            initial_weights[i] =lasso_coordinate_descent_step(i,feature_matrix,output,initial_weights,l1_penalty)
            change = abs(old_weights_i - initial_weights[i])
            #print 'change:' + str(change)
            weights_diff.append(change)
        if(sum(weights_diff) < tolerance):
            converged=True
    return initial_weights
        

Using the following parameters, learn the weights on the sales dataset. 

In [86]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0


First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [71]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [79]:
simple_feature_matrix

array([[  1.00000000e+00,   1.18000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   2.57000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   7.70000000e+02,   2.00000000e+00],
       ..., 
       [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00],
       [  1.00000000e+00,   1.60000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00]])

In [80]:
print normalized_simple_feature_matrix.shape
print simple_norms
normalized_simple_feature_matrix

(21613, 3)
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]


array([[ 0.00680209,  0.00353021,  0.00583571],
       [ 0.00680209,  0.00768869,  0.00583571],
       [ 0.00680209,  0.00230361,  0.00389048],
       ..., 
       [ 0.00680209,  0.00305154,  0.00389048],
       [ 0.00680209,  0.00478673,  0.00583571],
       [ 0.00680209,  0.00305154,  0.00389048]])

Then, run your implementation of LASSO coordinate descent:

In [84]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

change:0.421521123499
change:0.385601237416
change:0.0


In [73]:
weights

array([ 21624996.54143479,  63157248.45487124,         0.        ])

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [87]:
prediction = predict_output(normalized_simple_feature_matrix, weights)
RSS = ((prediction-output)**2).sum()

print RSS


1.63049246093e+15


In [ ]:
#2.Which features had weight zero at convergence?
# above o/p array([ 21624996.54143479,  63157248.45487124,         0.        ]) -. 2nd feature i.e bedrooms

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [88]:
train_data,test_data = sales.random_split(.8,seed=0)

In [116]:
train_data.head(1)

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	int
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float
	constant	int

Rows: 1

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|       0       |   1955   |      0       |  98178  | 47.51123398 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
+---------------+---------------+-----+
[1 rows x 22 columns]

Let us consider the following set of features.

In [89]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [129]:
my_output = 'price'
(train_feature_matrix, train_output) = get_numpy_data(train_data, all_features, my_output)
(normalized_train_feature_matrix, train_norms) = normalize_features(train_feature_matrix) # normalize features

In [98]:
print train_feature_matrix.shape
train_feature_matrix[0]

(17384, 14)


array([  1.00000000e+00,   3.00000000e+00,   1.00000000e+00,
         1.18000000e+03,   5.65000000e+03,   1.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   3.00000000e+00,
         7.00000000e+00,   1.18000000e+03,   0.00000000e+00,
         1.95500000e+03,   0.00000000e+00])

In [96]:
print train_norms
normalized_train_feature_matrix[0]

[  1.31848398e+02   4.60040216e+02   2.96850552e+02   2.99962419e+05
   5.81709718e+06   2.04107815e+02   1.15325626e+01   1.05933942e+02
   4.57793622e+02   1.02101959e+03   2.59726472e+05   7.01224951e+04
   2.59922094e+05   5.36953839e+04]


array([ 0.00758447,  0.00652117,  0.0033687 ,  0.00393383,  0.00097127,
        0.00489937,  0.        ,  0.        ,  0.00655317,  0.00685589,
        0.00454324,  0.        ,  0.00752148,  0.        ])

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [126]:
l1_penalty=1e7
initial_weights=np.zeros(14)
tolerance=1.
weights1e7 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)

In [127]:
weights1e7

array([ 24429598.34220436,         0.        ,         0.        ,
        48389176.48719848,         0.        ,         0.        ,
         3317511.16568974,   7329961.84498793,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [128]:
zip(weights1e7,all_features)

[(24429598.342204362, 'bedrooms'),
 (0.0, 'bathrooms'),
 (0.0, 'sqft_living'),
 (48389176.487198479, 'sqft_lot'),
 (0.0, 'floors'),
 (0.0, 'waterfront'),
 (3317511.1656897394, 'view'),
 (7329961.8449879326, 'condition'),
 (0.0, 'grade'),
 (0.0, 'sqft_above'),
 (0.0, 'sqft_basement'),
 (0.0, 'yr_built'),
 (0.0, 'yr_renovated')]

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [120]:
l1_penalty=1e8
initial_weights=np.zeros(14)
tolerance=1.
weights1e8 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)

In [121]:
weights1e8

array([ 71114625.75280938,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [122]:
zip(weights1e8,all_features)

[(71114625.752809376, 'bedrooms'),
 (0.0, 'bathrooms'),
 (0.0, 'sqft_living'),
 (0.0, 'sqft_lot'),
 (0.0, 'floors'),
 (0.0, 'waterfront'),
 (0.0, 'view'),
 (0.0, 'condition'),
 (0.0, 'grade'),
 (0.0, 'sqft_above'),
 (0.0, 'sqft_basement'),
 (0.0, 'yr_built'),
 (0.0, 'yr_renovated')]

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [123]:
l1_penalty=1e4
initial_weights=np.zeros(14)
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)

In [124]:
weights1e4

array([  1.07452054e+08,  -1.90511244e+07,   3.42603262e+06,
         1.73622706e+08,  -1.79305042e+06,  -4.47779699e+06,
         6.72209311e+06,   5.76010113e+06,   1.81160538e+07,
         9.74183215e+07,  -9.85424762e+07,  -2.60507297e+07,
        -1.88443642e+08,   3.03904311e+06])

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [125]:
zip(weights1e4,all_features)
# all are non-zero

[(107452053.54596432, 'bedrooms'),
 (-19051124.401710015, 'bathrooms'),
 (3426032.6154105961, 'sqft_living'),
 (173622706.42619511, 'sqft_lot'),
 (-1793050.4170872665, 'floors'),
 (-4477796.986295443, 'waterfront'),
 (6722093.110743029, 'view'),
 (5760101.125740896, 'condition'),
 (18116053.81542759, 'grade'),
 (97418321.512999117, 'sqft_above'),
 (-98542476.156629235, 'sqft_basement'),
 (-26050729.650327533, 'yr_built'),
 (-188443641.68728375, 'yr_renovated')]

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [131]:
normalized_weights1e4 = weights1e4 / train_norms
normalized_weights1e7 = weights1e7 / train_norms
normalized_weights1e8 = weights1e8 / train_norms


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [132]:
print normalized_weights1e7[3]

161.317463366


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [133]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [139]:
prediction=predict_output(test_feature_matrix,normalized_weights1e8)
RSS1 = ((prediction-test_output)**2).sum()
RSS1

537166150034084.88

In [140]:
prediction=predict_output(test_feature_matrix,normalized_weights1e7)
RSS2 = ((prediction-test_output)**2).sum()
RSS2

275962075771408.41

In [141]:
prediction=predict_output(test_feature_matrix,normalized_weights1e4)
RSS3 = ((prediction-test_output)**2).sum()
RSS3

206896417445794.34

***QUIZ QUESTION***

Which model performed best on the test data?

In [142]:
min(RSS1,RSS2,RSS3)

206896417445794.34